In [21]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
#from procurement_lib import send_slack_notification
#from procurement_lib import redash
    #redash and slack conections 
from analysts_tools.redash_methods import *
#from slack_notification import send_slack_notification
#from slack_notification import send_slack_notification
    #work with str
import ast
from procurement_lib import GoogleSheet, logging
    
from tabulate import tabulate
    #environment parameteres
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
def getchannel(city):
    if city in ['BOG', 'BAQ', 'MDE']:
        return ':flag-co:'
    elif city in ['CMX', 'GDL', 'PBC', 'MTY']:
        return ':flag-mx:'
    else:
        return ':flag-br:'

In [23]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
bs.site_identifier_value AS region_code,
bo.offer_id,
bo.offer_name AS discount,
DATE(bo.start_date) AS start_date,
DATE(bo.end_date) AS end_date,
(bo.offer_value::float)/100.00 as Value_Discount,
replace(
    substring(
        boic.order_item_match_rule, 
        position('["' in boic.order_item_match_rule) + 2, 
        position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
), '"', '') AS values_string

FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc

WHERE offer_name NOT ILIKE '%merma%'
AND offer_name NOT ILIKE 'ACMKTPLC%'
AND offer_name NOT ILIKE '%\_NB\_%'
AND offer_name NOT ILIKE 'BIG LIQUIDATION%'
AND DATE(bo.end_date) >= DATE_TRUNC('day', date(getdate()) + interval '1 day')
AND DATE(bo.start_date) <= DATE_TRUNC('day', date(getdate()) + interval '0 day')
--AND start_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
AND bo.offer_discount_type = 'PERCENT_OFF'
AND bo.offer_type = 'ORDER_ITEM'
AND bo.sndbx_id is NULL
AND (bo.archived is NULL OR bo.archived='N')
AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
AND bo.automatically_added = 'true'
), 

number_series AS (
SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
ed.region_code,
ed.offer_id,
ed.discount,
ed.start_date,
ed.end_date,
ed.value_discount,
bs.addl_product_id AS card_id

FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
WHERE
-- FIX SUPER DESCUENTOS
bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [24]:
query = """
SELECT DISTINCT
    p.product_id,
    bs.addl_product_id AS card_id,
    p.name,
    bs.name AS card_name
FROM postgres_main_co."purchase_orders.products" p
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs ON bs.upc = p.sku
WHERE p.deleted_at is null
 AND p.parent_id is null
 AND p.product_category_id IN (5,6,7,8,9,10,11,13,18)
 """
df_ids = read_connection_data_warehouse.runQuery(query)

In [25]:
df_final = df_offers.merge(df_ids, left_on=['card_id'], right_on=['card_id'], how='inner')

In [26]:
skus = df_final.product_id.unique()
len(skus)
skus = ",".join(str(num) for num in skus)

In [27]:
df_eagle = get_fresh_query_result("https://internal-redash.federate.frubana.com/",162844,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{"product_id": skus},20)

200


## PRODUCTS ALL 

In [28]:
len(df_eagle.product_id.unique())

362

## PRODUCTS OK

In [29]:
result = df_eagle.groupby('product_id')['fit_for_discount'].apply(lambda x: (x == 'FALSE').sum()).reindex(df_eagle['product_id'].unique(), fill_value=0).reset_index()

In [30]:
sku_alert_safe = result.loc[result.fit_for_discount == 0].product_id.unique()

In [31]:
len(sku_alert_safe)

167

## PRODUCTS ALL BAD

In [32]:
result2 = df_eagle.groupby('product_id')['fit_for_discount'].apply(lambda x: ((x == 'check_with_supply') | (x == 'TRUE')).sum()).reindex(df_eagle['product_id'].unique(), fill_value=0).reset_index()

In [33]:
sku_alert_bad = result2.loc[result2.fit_for_discount == 0].product_id.unique()

In [34]:
len(sku_alert_bad) 

120

## OTHER INFO

In [35]:
df_fin_eagle = df_eagle.loc[(~df_eagle.product_id.isin(sku_alert_safe))].reset_index(drop=True) # (~df_eagle.product_id.isin(sku_alert_bad)) & 

In [36]:
df_f =  df_fin_eagle.merge(df_final[['product_id','name']].drop_duplicates(), left_on=['product_id'], right_on=['product_id'], how='inner')

In [37]:
# Group by product_id and calculate average clearence_days
grouped_df = df_fin_eagle.groupby('product_id')['clearence_days'].mean().reset_index()

# Create the check column based on conditions
grouped_df['check'] = grouped_df['clearence_days'].apply(lambda x: 'FALSE' if x < 2 else ('check_with_supply' if x >= 2 and x < 4 else 'TRUE'))

In [38]:
df_f = grouped_df.loc[grouped_df.check.isin(['FALSE','check_with_supply'])].merge(df_final[['region_code','product_id','name','offer_id','discount','value_discount']].drop_duplicates(), left_on=['product_id'], right_on=['product_id'], how='inner')

In [39]:
def determine_responsable(discount):
    discount = discount.lower()  # Convert discount string to lowercase for case-insensitive comparison
    
    if "big liquidation" in discount or "acmktplc" in discount:
        return "MKTPLC"
    elif "hook" in discount:
        return "MKTPLC&DCT"
    else:
        return "DISCOUNTS"

df_f['responsable'] = df_f['discount'].apply(determine_responsable)
df_f['value_discount'] = (df_f['value_discount'] * 100).round(2).astype(str) + '%'

In [40]:
for city in df_f.region_code.unique():
    df_city = df_f[df_f.region_code == city]
    for  check in df_city.check.unique():
        df_check = df_city[df_city.check == check]
        if check == 'FALSE':
            word = 'alert'
        else:
            word = 'advertencia'
        for responsable in df_check.responsable.unique():
            df_responsable = df_check[df_check.responsable == responsable]
            
            a=tabulate(df_responsable[['region_code','product_id','name','offer_id','discount','value_discount']], 
                showindex=False, 
                headers=['region_code','product_id','name','offer_id','discount','value'])
    
            message = f"""
:{word}: *_ PRODUCTS ON STOCKOUT RISK for {city} _* {getchannel(city)}:
--> LEVEL: {check} / RESPONSABLE: {responsable} 
```
{a}
```
"""
            print(message)


:alert: *_ PRODUCTS ON STOCKOUT RISK for CMX _* :flag-mx::
--> LEVEL: FALSE / RESPONSABLE: MKTPLC&DCT 
```
region_code      product_id  name                                                                                                                   offer_id  discount                                                                        value
-------------  ------------  -------------------------------------------------------------------------------------------------------------------  ----------  ------------------------------------------------------------------------------  -------
CMX                     272  Arándano Seco  Bolsa al Granel desde 250g - Desde 250g                                                                -14509195  Hook_CMX_Arándano_Seco_x_250_gr_2_%May_20_2023                                  2.0%
CMX                     288  Chile Guajillo  Bolsa al Granel desde 250g - Desde 250g                                                               -14509513  H

In [41]:
#     ------------------------- INFO FINAL -------------------------------------------------
#     ------------------------- INFO FINAL -------------------------------------------------
#     ------------------------- INFO FINAL -------------------------------------------------
#    environment = os.environ.get('environment', 'DEV_SAMPLE')
#    people = {'ALL': ["felipe"]}.get('ALL', [])
#    send_slack_notification('ALERT_PRICEFRU', message, people, ':papaya:')

In [42]:
df_f.to_excel('prueva.xlsx')